In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

train_df = pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")
test_df = pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")
sub = pd.read_csv("/kaggle/input/cat-in-the-dat/sample_submission.csv")

train = train_df.drop(['target'], axis=1)
train_target = train_df['target'].to_numpy()

train = train.drop(['id'], axis=1)
test = test_df.drop(['id'],axis=1)


In [ ]:
type(train["nom_9"][2])

In [ ]:
data = pd.concat([train, test])
#data =train.append(test, sort=False)
data

In [ ]:
drop_cols=["bin_0"]

# Split 2 Letters; This is the only part which is not generic and would actually require data inspection
data["ord_5a"]=data["ord_5"].str[0]
data["ord_5b"]=data["ord_5"].str[1]

drop_cols.append("ord_5")

In [ ]:
data = data.drop(drop_cols, axis = 1)

data

In [ ]:
for col in ["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"]:
    train_vals = set(train[col].unique())
    test_vals = set(test[col].unique())
    xor_cat_vals=train_vals ^ test_vals # xor ,find elements that is in only 1 side
    print(list(xor_cat_vals))
    if xor_cat_vals:
        data.replace(list(xor_cat_vals), "ddd")
            

In [ ]:
data["nom_9"][1]

In [ ]:
for i in list(set(data["nom_9"].unique())):
    if i == 'ddd':
        print("found")

In [ ]:
# for one hot encoding everything!

columns = [i for i in data.columns]
dummies = pd.get_dummies(data,columns=columns, drop_first=True,sparse=True)

In [ ]:
%%time
train = dummies.iloc[:train.shape[0], :]
test = dummies.iloc[train.shape[0]:, :]

print(train)
print(test)

In [ ]:
train = train.sparse.to_coo().tocsr()
test = test.sparse.to_coo().tocsr()

In [ ]:
##### %%time

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression

# Model
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model', split=5):
    kf = KFold(n_splits=split)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/', split)
        dev_X, val_X = train[dev_index], train[val_index]
        dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score))
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    pred_full_test = pred_full_test / split
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores}
    return results


def runLR(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = LogisticRegression(**params)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = model.predict_proba(test_X)[:, 1]
    print('Predict 2/2')
    pred_test_y2 = model.predict_proba(test_X2)[:, 1]
    return pred_test_y, pred_test_y2




In [ ]:
lr_params = {'solver': 'lbfgs', 'C': 0.1, 'max_iter': 1000}
results = run_cv_model(train, test, train_target, runLR, lr_params, auc, 'lr', split=10)

In [ ]:
test_id = sub["id"]

In [ ]:
submission = pd.DataFrame({'id': test_id, 'target': results['test']})
submission.to_csv('submission_kfold_max.csv', index=False)